In [1]:
import sys, os, itertools, sklearn
sys.path.append(os.getcwd())
import pandas as pd
import numpy as np
import _pickle as pkl
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True, edgeitems=10)
np.core.arrayprint._line_width = 80

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
from causal_boosting import CausalBoosting
from naive_pooling import NaivePooling
from base_environment import BaseEnvironment
from collection_environment import CollectionEnvironment
from regression_method import PolynomialRegression, RandomForestRegression, DecisionTreeRegression


In [2]:
n_rep = 20
n_samples_list= [200,500,1000,5000,10000]

In [3]:
def printresults(results, n_samples_list):
    for i in np.arange(len(results)):
        print(f"Sample Size: {n_samples_list[i]:5d}: Loss: {results[i][0]:4.3f}pm{results[i][1]:4.3f}")

## Function A

In [4]:
ik_h, se_h = [], (lambda input_samples:input_samples)
ik_x1, se_x1 = ['h'], (lambda input_samples: 2*input_samples[0] + input_samples[1])
ik_x2, se_x2 = ['h', 'x1'], (lambda input_samples: input_samples[0] + input_samples[1] + input_samples[2])
ik_x3, se_x3 = ['x1', 'x2'], (lambda input_samples: -input_samples[0] * 2*input_samples[1] + input_samples[2])
ik_x4, se_x4 = ['x1', 'x3'], (lambda input_samples: np.log(1+np.abs(input_samples[0])) + input_samples[1] + input_samples[2])

ik_y, se_y = ['h','x1','x2','x3','x4'], (lambda input_samples:
                                        2*input_samples[0] 
                                         + (input_samples[1]>0)*1.
                                        + (input_samples[2]>0)*1.
                                         - (input_samples[2]>0)*(input_samples[3]<-1)*2.
                                         + (input_samples[1]<0)*(input_samples[4]<-1)*2.
                                         + (input_samples[1]<0)*(input_samples[2]<1)*(input_samples[3]<-1)*3.
                                         + input_samples[5])
ik_x5, se_x5 = ['x2', 'x4', 'y'], (lambda input_samples: 2*input_samples[0] + input_samples[1] - input_samples[2] + input_samples[3])

fh = {'input_keys':ik_h, 'structural_eq': se_h}
f1 = {'input_keys':ik_x1, 'structural_eq': se_x1}
f2 = {'input_keys':ik_x2, 'structural_eq': se_x2}
f3 = {'input_keys':ik_x3, 'structural_eq': se_x3}
f4 = {'input_keys':ik_x4, 'structural_eq': se_x4}
f5 = {'input_keys':ik_x5, 'structural_eq': se_x5}
fy = {'input_keys':ik_y, 'structural_eq': se_y}

sh = lambda n_samples: np.random.normal(size=n_samples)
s1 = lambda n_samples: np.random.normal(size=n_samples)
s2 = lambda n_samples: np.random.normal(size=n_samples)
s3 = lambda n_samples: np.random.normal(size=n_samples)
s4 = lambda n_samples: np.random.normal(size=n_samples)
s5 = lambda n_samples: np.random.normal(size=n_samples)
sy = lambda n_samples: np.random.normal(size=n_samples)

topo_order = ['h','x1','x2','x3','x4','y','x5']
y_key = 'y'
x_key = ['x1','x2','x3','x4','x5']

structural_equation_dict = {'x1':f1, 'x2':f2, 'x3':f3, 'y':fy, 'x4':f4, 'x5':f5, 'h':fh}
disturbance_sampler_dict = {'x1':s1, 'x2':s2, 'x3':s3, 'y':sy, 'x4':s4, 'x5':s5, 'h':sh}

base = BaseEnvironment(structural_equation_dict, disturbance_sampler_dict, topo_order, y_key, x_key)
coll_env = CollectionEnvironment(base)

#### Case A1

In [5]:
boosting_oracle_loss, boosting_validation_loss, naive_oracle_loss, naive_validation_loss = [], [], [], []

for n_samples in n_samples_list:
    boosting_oracle_loss_list=[]
    boosting_validation_loss_list=[]
    naive_oracle_loss_list=[]
    naive_validation_loss_list=[]
    for i in np.arange(n_rep):
        coll_env.reset_env()
        coll_env.add_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        coll_env.add_env('e2', {'x1':{'type':'independent'},'x4':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)

        boosting = CausalBoosting(DecisionTreeRegression,
                              max_n_iter=10,
                              params_method={},
                             )
        naive = NaivePooling(DecisionTreeRegression,
                              params_method={},
                             )
        try:
            boosting.fitCV(coll_env)
            boosting_oracle_loss_list.append(boosting.evaluate_oracle())
            boosting_validation_loss_list.append(boosting.evaluate_validation())
            
            naive.fitCV(coll_env)
            naive_oracle_loss_list.append(naive.evaluate_oracle())
            naive_validation_loss_list.append(naive.evaluate_validation())
        except:
            print('Raised an error.')
            pass
        
    boosting_oracle_loss.append((np.mean(boosting_oracle_loss_list),np.std(boosting_oracle_loss_list)))
    boosting_validation_loss.append((np.mean(boosting_validation_loss_list),np.std(boosting_validation_loss_list)))
    naive_oracle_loss.append((np.mean(naive_oracle_loss_list),np.std(naive_oracle_loss_list)))
    naive_validation_loss.append((np.mean(naive_validation_loss_list),np.std(naive_validation_loss_list)))

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

...

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL, no acceleration applied.
WARN: aa_init returned NULL,

In [6]:
print('Causal Gradient Aggregation - Oracle loss')
printresults(boosting_oracle_loss, n_samples_list)
print('Causal Gradient Aggregation - Validation loss')
printresults(boosting_validation_loss, n_samples_list)
print('Naive Method - Oracle loss')
printresults(naive_oracle_loss, n_samples_list)
print('Naive Method - Validation loss')
printresults(naive_validation_loss, n_samples_list)

Causal Gradient Aggregation - Oracle loss
Sample Size:   200: Loss: 1.278pm0.259
Sample Size:   500: Loss: 1.170pm0.369
Sample Size:  1000: Loss: 0.801pm0.489
Sample Size:  5000: Loss: 0.229pm0.097
Sample Size: 10000: Loss: 0.105pm0.033
Causal Gradient Aggregation - Validation loss
Sample Size:   200: Loss: 6.547pm0.824
Sample Size:   500: Loss: 6.128pm0.501
Sample Size:  1000: Loss: 5.675pm0.389
Sample Size:  5000: Loss: 5.295pm0.176
Sample Size: 10000: Loss: 5.190pm0.105
Naive Method - Oracle loss
Sample Size:   200: Loss: 4.271pm0.640
Sample Size:   500: Loss: 4.335pm0.410
Sample Size:  1000: Loss: 4.373pm0.218
Sample Size:  5000: Loss: 4.272pm0.107
Sample Size: 10000: Loss: 4.217pm0.060
Naive Method - Validation loss
Sample Size:   200: Loss: 2.665pm0.418
Sample Size:   500: Loss: 2.084pm0.166
Sample Size:  1000: Loss: 1.741pm0.107
Sample Size:  5000: Loss: 1.352pm0.041
Sample Size: 10000: Loss: 1.311pm0.031


#### Case A2

In [ ]:
boosting_oracle_loss, boosting_validation_loss, naive_oracle_loss, naive_validation_loss = [], [], [], []

for n_samples in n_samples_list:
    boosting_oracle_loss_list=[]
    boosting_validation_loss_list=[]
    naive_oracle_loss_list=[]
    naive_validation_loss_list=[]
    for i in np.arange(n_rep):
        coll_env.reset_env()
        #coll_env.add_synthetic_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        coll_env.add_env('e2', {'x1':{'type':'independent'},'x4':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)

        boosting = CausalBoosting(DecisionTreeRegression,
                              max_n_iter=10,
                              params_method={},
                             )
        naive = NaivePooling(DecisionTreeRegression,
                              params_method={},
                             )
        try:
            boosting.fitCV(coll_env)
            boosting_oracle_loss_list.append(boosting.evaluate_oracle())
            boosting_validation_loss_list.append(boosting.evaluate_validation())
            
            naive.fitCV(coll_env)
            naive_oracle_loss_list.append(naive.evaluate_oracle())
            naive_validation_loss_list.append(naive.evaluate_validation())
        except:
            print('Raised an error.')
            pass
        
    boosting_oracle_loss.append((np.mean(boosting_oracle_loss_list),np.std(boosting_oracle_loss_list)))
    boosting_validation_loss.append((np.mean(boosting_validation_loss_list),np.std(boosting_validation_loss_list)))
    naive_oracle_loss.append((np.mean(naive_oracle_loss_list),np.std(naive_oracle_loss_list)))
    naive_validation_loss.append((np.mean(naive_validation_loss_list),np.std(naive_validation_loss_list)))

In [15]:
print('Causal Gradient Aggregation - Oracle loss')
printresults(boosting_oracle_loss, n_samples_list)
print('Causal Gradient Aggregation - Validation loss')
printresults(boosting_validation_loss, n_samples_list)
print('Naive Method - Oracle loss')
printresults(naive_oracle_loss, n_samples_list)
print('Naive Method - Validation loss')
printresults(naive_validation_loss, n_samples_list)

Causal Gradient Aggregation - Oracle loss
Sample Size:   200: Loss: 6.820pm1.422
Sample Size:   500: Loss: 5.990pm0.670
Sample Size:  1000: Loss: 5.297pm0.520
Sample Size:  5000: Loss: 4.930pm0.231
Sample Size: 10000: Loss: 4.860pm0.106
Causal Gradient Aggregation - Validation loss
Sample Size:   200: Loss: 19.563pm4.327
Sample Size:   500: Loss: 17.702pm2.065
Sample Size:  1000: Loss: 17.082pm1.924
Sample Size:  5000: Loss: 17.213pm0.577
Sample Size: 10000: Loss: 17.066pm0.405
Naive Method - Oracle loss
Sample Size:   200: Loss: 6.043pm1.726
Sample Size:   500: Loss: 5.372pm0.783
Sample Size:  1000: Loss: 4.815pm0.596
Sample Size:  5000: Loss: 4.534pm0.274
Sample Size: 10000: Loss: 4.307pm0.155
Naive Method - Validation loss
Sample Size:   200: Loss: 3.197pm1.004
Sample Size:   500: Loss: 2.313pm0.291
Sample Size:  1000: Loss: 1.991pm0.213
Sample Size:  5000: Loss: 1.472pm0.059
Sample Size: 10000: Loss: 1.301pm0.040


#### Case A3

In [ ]:
boosting_oracle_loss, boosting_validation_loss, naive_oracle_loss, naive_validation_loss = [], [], [], []

for n_samples in n_samples_list:
    boosting_oracle_loss_list=[]
    boosting_validation_loss_list=[]
    naive_oracle_loss_list=[]
    naive_validation_loss_list=[]
    for i in np.arange(n_rep):
        coll_env.reset_env()
        #coll_env.add_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        #coll_env.add_env('e2', {'x1':{'type':'independent'},'x4':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)

        boosting = CausalBoosting(DecisionTreeRegression,
                              max_n_iter=10,
                              params_method={},
                             )
        naive = NaivePooling(DecisionTreeRegression,
                              params_method={},
                             )
        try:
            boosting.fitCV(coll_env)
            boosting_oracle_loss_list.append(boosting.evaluate_oracle())
            boosting_validation_loss_list.append(boosting.evaluate_validation())
            
            naive.fitCV(coll_env)
            naive_oracle_loss_list.append(naive.evaluate_oracle())
            naive_validation_loss_list.append(naive.evaluate_validation())
        except:
            print('Raised an error.')
            pass
        
    boosting_oracle_loss.append((np.mean(boosting_oracle_loss_list),np.std(boosting_oracle_loss_list)))
    boosting_validation_loss.append((np.mean(boosting_validation_loss_list),np.std(boosting_validation_loss_list)))
    naive_oracle_loss.append((np.mean(naive_oracle_loss_list),np.std(naive_oracle_loss_list)))
    naive_validation_loss.append((np.mean(naive_validation_loss_list),np.std(naive_validation_loss_list)))

In [14]:
print('Causal Gradient Aggregation - Oracle loss')
printresults(boosting_oracle_loss, n_samples_list)
print('Causal Gradient Aggregation - Validation loss')
printresults(boosting_validation_loss, n_samples_list)
print('Naive Method - Oracle loss')
printresults(naive_oracle_loss, n_samples_list)
print('Naive Method - Validation loss')
printresults(naive_validation_loss, n_samples_list)

Causal Gradient Aggregation - Oracle loss
Sample Size:   200: Loss: 6.820pm1.422
Sample Size:   500: Loss: 5.990pm0.670
Sample Size:  1000: Loss: 5.297pm0.520
Sample Size:  5000: Loss: 4.930pm0.231
Sample Size: 10000: Loss: 4.860pm0.106
Causal Gradient Aggregation - Validation loss
Sample Size:   200: Loss: 19.563pm4.327
Sample Size:   500: Loss: 17.702pm2.065
Sample Size:  1000: Loss: 17.082pm1.924
Sample Size:  5000: Loss: 17.213pm0.577
Sample Size: 10000: Loss: 17.066pm0.405
Naive Method - Oracle loss
Sample Size:   200: Loss: 6.043pm1.726
Sample Size:   500: Loss: 5.372pm0.783
Sample Size:  1000: Loss: 4.815pm0.596
Sample Size:  5000: Loss: 4.534pm0.274
Sample Size: 10000: Loss: 4.307pm0.155
Naive Method - Validation loss
Sample Size:   200: Loss: 3.197pm1.004
Sample Size:   500: Loss: 2.313pm0.291
Sample Size:  1000: Loss: 1.991pm0.213
Sample Size:  5000: Loss: 1.472pm0.059
Sample Size: 10000: Loss: 1.301pm0.040


## Function B

In [9]:
ik_h, se_h = [], (lambda input_samples:input_samples)
ik_x1, se_x1 = ['h'], (lambda input_samples: 2*input_samples[0] + input_samples[1])
ik_x2, se_x2 = ['h', 'x1'], (lambda input_samples: input_samples[0] + input_samples[1] + input_samples[2])
ik_x3, se_x3 = ['x1', 'x2'], (lambda input_samples: -input_samples[0] * 2*input_samples[1] + input_samples[2])
ik_x4, se_x4 = ['x1', 'x3'], (lambda input_samples: np.log(1+np.abs(input_samples[0])) + input_samples[1] + input_samples[2])

ik_y, se_y = ['h','x1','x2','x3','x4'], (lambda input_samples:
                                        2*input_samples[0] 
                                         + input_samples[1]
                                        - input_samples[2]
                                         - (input_samples[2]>0)*(input_samples[3]<-1)*5.
                                         + (input_samples[1]<0)*(input_samples[4]<-1)*2.
                                         + (input_samples[1]<0)*(input_samples[2]<1)*(input_samples[3]<-1)*3.
                                         + input_samples[5])
ik_x5, se_x5 = ['x2', 'x4', 'y'], (lambda input_samples: 2*input_samples[0] + input_samples[1] - input_samples[2] + input_samples[3])

fh = {'input_keys':ik_h, 'structural_eq': se_h}
f1 = {'input_keys':ik_x1, 'structural_eq': se_x1}
f2 = {'input_keys':ik_x2, 'structural_eq': se_x2}
f3 = {'input_keys':ik_x3, 'structural_eq': se_x3}
f4 = {'input_keys':ik_x4, 'structural_eq': se_x4}
f5 = {'input_keys':ik_x5, 'structural_eq': se_x5}
fy = {'input_keys':ik_y, 'structural_eq': se_y}

sh = lambda n_samples: np.random.normal(size=n_samples)
s1 = lambda n_samples: np.random.normal(size=n_samples)
s2 = lambda n_samples: np.random.normal(size=n_samples)
s3 = lambda n_samples: np.random.normal(size=n_samples)
s4 = lambda n_samples: np.random.normal(size=n_samples)
s5 = lambda n_samples: np.random.normal(size=n_samples)
sy = lambda n_samples: np.random.normal(size=n_samples)

topo_order = ['h','x1','x2','x3','x4','y','x5']
y_key = 'y'
x_key = ['x1','x2','x3','x4','x5']

structural_equation_dict = {'x1':f1, 'x2':f2, 'x3':f3, 'y':fy, 'x4':f4, 'x5':f5, 'h':fh}
disturbance_sampler_dict = {'x1':s1, 'x2':s2, 'x3':s3, 'y':sy, 'x4':s4, 'x5':s5, 'h':sh}

base = BaseEnvironment(structural_equation_dict, disturbance_sampler_dict, topo_order, y_key, x_key)
coll_env = CollectionEnvironment(base)

#### Case B1

In [ ]:
boosting_oracle_loss, boosting_validation_loss, naive_oracle_loss, naive_validation_loss = [], [], [], []

for n_samples in n_samples_list:
    boosting_oracle_loss_list=[]
    boosting_validation_loss_list=[]
    naive_oracle_loss_list=[]
    naive_validation_loss_list=[]
    for i in np.arange(n_rep):
        coll_env.reset_env()
        coll_env.add_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        coll_env.add_env('e2', {'x1':{'type':'independent'},'x4':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)

        boosting = CausalBoosting(DecisionTreeRegression,
                              max_n_iter=10,
                              params_method={},
                             )
        naive = NaivePooling(DecisionTreeRegression,
                              params_method={},
                             )
        try:
            boosting.fitCV(coll_env)
            boosting_oracle_loss_list.append(boosting.evaluate_oracle())
            boosting_validation_loss_list.append(boosting.evaluate_validation())
            
            naive.fitCV(coll_env)
            naive_oracle_loss_list.append(naive.evaluate_oracle())
            naive_validation_loss_list.append(naive.evaluate_validation())
        except:
            print('Raised an error.')
            pass
        
    boosting_oracle_loss.append((np.mean(boosting_oracle_loss_list),np.std(boosting_oracle_loss_list)))
    boosting_validation_loss.append((np.mean(boosting_validation_loss_list),np.std(boosting_validation_loss_list)))
    naive_oracle_loss.append((np.mean(naive_oracle_loss_list),np.std(naive_oracle_loss_list)))
    naive_validation_loss.append((np.mean(naive_validation_loss_list),np.std(naive_validation_loss_list)))

In [13]:
print('Causal Gradient Aggregation - Oracle loss')
printresults(boosting_oracle_loss, n_samples_list)
print('Causal Gradient Aggregation - Validation loss')
printresults(boosting_validation_loss, n_samples_list)
print('Naive Method - Oracle loss')
printresults(naive_oracle_loss, n_samples_list)
print('Naive Method - Validation loss')
printresults(naive_validation_loss, n_samples_list)

Causal Gradient Aggregation - Oracle loss
Sample Size:   200: Loss: 6.820pm1.422
Sample Size:   500: Loss: 5.990pm0.670
Sample Size:  1000: Loss: 5.297pm0.520
Sample Size:  5000: Loss: 4.930pm0.231
Sample Size: 10000: Loss: 4.860pm0.106
Causal Gradient Aggregation - Validation loss
Sample Size:   200: Loss: 19.563pm4.327
Sample Size:   500: Loss: 17.702pm2.065
Sample Size:  1000: Loss: 17.082pm1.924
Sample Size:  5000: Loss: 17.213pm0.577
Sample Size: 10000: Loss: 17.066pm0.405
Naive Method - Oracle loss
Sample Size:   200: Loss: 6.043pm1.726
Sample Size:   500: Loss: 5.372pm0.783
Sample Size:  1000: Loss: 4.815pm0.596
Sample Size:  5000: Loss: 4.534pm0.274
Sample Size: 10000: Loss: 4.307pm0.155
Naive Method - Validation loss
Sample Size:   200: Loss: 3.197pm1.004
Sample Size:   500: Loss: 2.313pm0.291
Sample Size:  1000: Loss: 1.991pm0.213
Sample Size:  5000: Loss: 1.472pm0.059
Sample Size: 10000: Loss: 1.301pm0.040


#### Case B2

In [ ]:
boosting_oracle_loss, boosting_validation_loss, naive_oracle_loss, naive_validation_loss = [], [], [], []

for n_samples in n_samples_list:
    boosting_oracle_loss_list=[]
    boosting_validation_loss_list=[]
    naive_oracle_loss_list=[]
    naive_validation_loss_list=[]
    for i in np.arange(n_rep):
        coll_env.reset_env()
        #coll_env.add_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        coll_env.add_env('e2', {'x1':{'type':'independent'},'x4':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)

        boosting = CausalBoosting(DecisionTreeRegression,
                              max_n_iter=10,
                              params_method={},
                             )
        naive = NaivePooling(DecisionTreeRegression,
                              params_method={},
                             )
        try:
            boosting.fitCV(coll_env)
            boosting_oracle_loss_list.append(boosting.evaluate_oracle())
            boosting_validation_loss_list.append(boosting.evaluate_validation())
            
            naive.fitCV(coll_env)
            naive_oracle_loss_list.append(naive.evaluate_oracle())
            naive_validation_loss_list.append(naive.evaluate_validation())
        except:
            print('Raised an error.')
            pass
        
    boosting_oracle_loss.append((np.mean(boosting_oracle_loss_list),np.std(boosting_oracle_loss_list)))
    boosting_validation_loss.append((np.mean(boosting_validation_loss_list),np.std(boosting_validation_loss_list)))
    naive_oracle_loss.append((np.mean(naive_oracle_loss_list),np.std(naive_oracle_loss_list)))
    naive_validation_loss.append((np.mean(naive_validation_loss_list),np.std(naive_validation_loss_list)))

In [16]:
print('Causal Gradient Aggregation - Oracle loss')
printresults(boosting_oracle_loss, n_samples_list)
print('Causal Gradient Aggregation - Validation loss')
printresults(boosting_validation_loss, n_samples_list)
print('Naive Method - Oracle loss')
printresults(naive_oracle_loss, n_samples_list)
print('Naive Method - Validation loss')
printresults(naive_validation_loss, n_samples_list)

Causal Gradient Aggregation - Oracle loss
Sample Size:   200: Loss: 6.820pm1.422
Sample Size:   500: Loss: 5.990pm0.670
Sample Size:  1000: Loss: 5.297pm0.520
Sample Size:  5000: Loss: 4.930pm0.231
Sample Size: 10000: Loss: 4.860pm0.106
Causal Gradient Aggregation - Validation loss
Sample Size:   200: Loss: 19.563pm4.327
Sample Size:   500: Loss: 17.702pm2.065
Sample Size:  1000: Loss: 17.082pm1.924
Sample Size:  5000: Loss: 17.213pm0.577
Sample Size: 10000: Loss: 17.066pm0.405
Naive Method - Oracle loss
Sample Size:   200: Loss: 6.043pm1.726
Sample Size:   500: Loss: 5.372pm0.783
Sample Size:  1000: Loss: 4.815pm0.596
Sample Size:  5000: Loss: 4.534pm0.274
Sample Size: 10000: Loss: 4.307pm0.155
Naive Method - Validation loss
Sample Size:   200: Loss: 3.197pm1.004
Sample Size:   500: Loss: 2.313pm0.291
Sample Size:  1000: Loss: 1.991pm0.213
Sample Size:  5000: Loss: 1.472pm0.059
Sample Size: 10000: Loss: 1.301pm0.040


#### Case B3

In [ ]:
boosting_oracle_loss, boosting_validation_loss, naive_oracle_loss, naive_validation_loss = [], [], [], []

for n_samples in n_samples_list:
    boosting_oracle_loss_list=[]
    boosting_validation_loss_list=[]
    naive_oracle_loss_list=[]
    naive_validation_loss_list=[]
    for i in np.arange(n_rep):
        coll_env.reset_env()
        #coll_env.add_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        #coll_env.add_env('e2', {'x1':{'type':'independent'},'x4':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)

        boosting = CausalBoosting(DecisionTreeRegression,
                              max_n_iter=10,
                              params_method={},
                             )
        naive = NaivePooling(DecisionTreeRegression,
                              params_method={},
                             )
        try:
            boosting.fitCV(coll_env)
            boosting_oracle_loss_list.append(boosting.evaluate_oracle())
            boosting_validation_loss_list.append(boosting.evaluate_validation())
            
            naive.fitCV(coll_env)
            naive_oracle_loss_list.append(naive.evaluate_oracle())
            naive_validation_loss_list.append(naive.evaluate_validation())
        except:
            print('Raised an error.')
            pass
        
    boosting_oracle_loss.append((np.mean(boosting_oracle_loss_list),np.std(boosting_oracle_loss_list)))
    boosting_validation_loss.append((np.mean(boosting_validation_loss_list),np.std(boosting_validation_loss_list)))
    naive_oracle_loss.append((np.mean(naive_oracle_loss_list),np.std(naive_oracle_loss_list)))
    naive_validation_loss.append((np.mean(naive_validation_loss_list),np.std(naive_validation_loss_list)))

In [17]:
print('Causal Gradient Aggregation - Oracle loss')
printresults(boosting_oracle_loss, n_samples_list)
print('Causal Gradient Aggregation - Validation loss')
printresults(boosting_validation_loss, n_samples_list)
print('Naive Method - Oracle loss')
printresults(naive_oracle_loss, n_samples_list)
print('Naive Method - Validation loss')
printresults(naive_validation_loss, n_samples_list)

Causal Gradient Aggregation - Oracle loss
Sample Size:   200: Loss: 6.820pm1.422
Sample Size:   500: Loss: 5.990pm0.670
Sample Size:  1000: Loss: 5.297pm0.520
Sample Size:  5000: Loss: 4.930pm0.231
Sample Size: 10000: Loss: 4.860pm0.106
Causal Gradient Aggregation - Validation loss
Sample Size:   200: Loss: 19.563pm4.327
Sample Size:   500: Loss: 17.702pm2.065
Sample Size:  1000: Loss: 17.082pm1.924
Sample Size:  5000: Loss: 17.213pm0.577
Sample Size: 10000: Loss: 17.066pm0.405
Naive Method - Oracle loss
Sample Size:   200: Loss: 6.043pm1.726
Sample Size:   500: Loss: 5.372pm0.783
Sample Size:  1000: Loss: 4.815pm0.596
Sample Size:  5000: Loss: 4.534pm0.274
Sample Size: 10000: Loss: 4.307pm0.155
Naive Method - Validation loss
Sample Size:   200: Loss: 3.197pm1.004
Sample Size:   500: Loss: 2.313pm0.291
Sample Size:  1000: Loss: 1.991pm0.213
Sample Size:  5000: Loss: 1.472pm0.059
Sample Size: 10000: Loss: 1.301pm0.040
